In [15]:
from datetime import datetime
import torch
import numpy as np
from pathlib import Path

from utils.settings.config import LINEAR_ENCODER, CROP_ENCODING
num_discrete_labels = len(set(LINEAR_ENCODER.values()))

def starttimer():
    start = datetime.now()
    print(f'{start.strftime("%Y-%m-%d %H:%M:%S")}')
    return start

def endtimer(start):
    end = datetime.now()
    print(f'{end.strftime("%Y-%m-%d %H:%M:%S")}')
    durn = end - start
    print(f'Duration: {durn.total_seconds()}s')
    return end, durn

In [16]:
run_path = "logs/unet/20220807233511/run_20220807233511/"
run_path = Path(run_path)

testrun_path = Path(run_path / f'testrun_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
testrun_path.mkdir(exist_ok=True, parents=True)

In [17]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), dtype=torch.uint8, device='cuda')
pred = torch.randint(11, (833504,), dtype=torch.uint8, device='cuda')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-15 14:24:34
2022-08-15 14:28:10
Duration: 216.570497s


In [18]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), device='cuda')
pred = torch.randint(11, (833504,), device='cuda')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-15 14:28:10
2022-08-15 14:31:08
Duration: 177.392271s


In [19]:
confusion_matrix = torch.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), dtype=torch.uint8, device='cpu')
pred = torch.randint(11, (833504,), dtype=torch.uint8, device='cpu')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-15 14:31:08
2022-08-15 14:31:53
Duration: 45.480613s


In [20]:
confusion_matrix = np.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (833504,), device='cpu')
pred = torch.randint(11, (833504,), device='cpu')
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-15 14:31:53
2022-08-15 14:31:58
Duration: 4.908068s


In [21]:
confusion_matrix = np.zeros([num_discrete_labels, num_discrete_labels])
label = torch.randint(11, (224,61,61,)).cpu().detach().numpy().flatten()
pred = torch.randint(11, (224,61,61,)).cpu().detach().numpy().flatten()
s = starttimer()
for i in range(label.shape[0]):
    confusion_matrix[label[i], pred[i]] += 1
e,d = endtimer(s)

2022-08-15 14:31:58
2022-08-15 14:31:59
Duration: 0.431001s


In [45]:
from sklearn.metrics import confusion_matrix
cm = np.zeros([num_discrete_labels-1, num_discrete_labels-1])

label = torch.randint(11, (224,61,61,)).flatten().cpu().detach()
pred = torch.randint(11, (224,61,61,)).flatten().cpu().detach()

s = starttimer()
cm_delta = confusion_matrix(label, pred)
cm = cm + cm_delta
e,d = endtimer(s)

2022-08-15 14:47:39
2022-08-15 14:47:40
Duration: 0.129001s


In [93]:
from torchmetrics.functional import dice

dice_score = []
for i in range(10):
    label = torch.randint(12, (224,61,61,)).flatten().cpu().detach()
    pred = torch.randint(12, (224,61,61,)).flatten().cpu().detach()
    step_dice_score = dice(pred, label, num_classes=12, multiclass=True, zero_division=1, ignore_index=0, average='none')
    dice_score.append(step_dice_score.numpy())

dice_score = np.array(dice_score)
dice_score = dice_score[:,1:] # Drop zero label

print(f"Dice Score\n{dice_score}")
print(np.average(dice_score, axis=0))


Dice Score
[[0.08472826 0.08319587 0.08372173 0.08160654 0.08206059 0.08184217
  0.0857697  0.08151936 0.08385992 0.0848389  0.08066557]
 [0.08181564 0.08483499 0.08249715 0.08256092 0.0841168  0.08364642
  0.08112432 0.08251867 0.08223022 0.08380257 0.08337298]
 [0.08486968 0.08468919 0.08197322 0.08191588 0.08375364 0.08491933
  0.08427685 0.08606374 0.08219178 0.08253139 0.08366011]
 [0.08298323 0.0835162  0.08266919 0.08312801 0.08460183 0.0838313
  0.08421264 0.08277788 0.08361756 0.08380831 0.08341897]
 [0.08200999 0.08340331 0.08356722 0.08300404 0.08395467 0.08366252
  0.08344851 0.08341793 0.08416447 0.08347618 0.08374469]
 [0.08369589 0.08297019 0.0834953  0.08414627 0.08523703 0.08357806
  0.08214142 0.08038588 0.08235685 0.08315092 0.08158075]
 [0.08432846 0.08295669 0.08302313 0.08250605 0.08260415 0.08417162
  0.08286054 0.08298139 0.08359385 0.0818174  0.08311132]
 [0.0821448  0.08299265 0.08432989 0.08301453 0.08072793 0.08352835
  0.08482324 0.08255092 0.08468756 0.083